In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os

In [2]:
def get_csv_data(path_logs, path_csv):
    data = pd.read_csv(path_csv, sep='\t')
    dfs = []
    for dirname, _, filenames in os.walk(path_logs):
        for filename in filenames:
            if filename.endswith('.csv'):
                file_path = os.path.join(dirname, filename).replace('\\', '/')
                df = pd.read_csv(file_path,sep=' ')
                
                row = data.loc[data['log_id'] == int(filename[:-4])] # https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
                # print(row)
                arr = row.values.tolist()
                arr.append(df)
                dfs.append(arr)
    return dfs

In [3]:
dfs = get_csv_data('../the-attentive-cursor-dataset/logs', '../the-attentive-cursor-dataset/groundtruth.tsv')

In [5]:
cleandf = []
for row in range(len(dfs)):
    cleandf.append([dfs[row][0][2],dfs[row][1]['event'].value_counts()])

print(cleandf)

[[5, scroll          61
mousemove       22
mouseover        9
mousedown        6
mouseup          5
blur             3
focus            3
click            2
load             1
contextmenu      1
beforeunload     1
unload           1
Name: event, dtype: int64], [2, mousemove       9
scroll          7
mouseover       3
load            1
mousedown       1
mouseup         1
click           1
beforeunload    1
Name: event, dtype: int64], [4, mousemove       39
scroll          11
mouseover        3
blur             2
load             1
focus            1
beforeunload     1
unload           1
Name: event, dtype: int64], [2, mousemove       33
scroll          17
mouseover       13
blur             4
focus            4
load             1
mousedown        1
mouseup          1
click            1
beforeunload     1
unload           1
Name: event, dtype: int64], [4, mousemove       9
mouseover       3
blur            2
focus           2
load            1
beforeunload    1
unload          1
Name: ev

In [23]:
tab = np.array(cleandf, dtype='object')
for row in tab :
    for input in ['mouseup','focus','load','beforeunload','unload','touchend'] :
        if input in row[1].keys() :
            row[1].pop(input)

[[5 scroll         61.0
    mousemove      22.0
    mouseover       9.0
    mousedown       6.0
    blur            3.0
    click           2.0
    contextmenu     1.0
    Name: event, dtype: float64]
 [2 mousemove    9.0
    scroll       7.0
    mouseover    3.0
    mousedown    1.0
    click        1.0
    Name: event, dtype: float64]
 [4 mousemove    39.0
    scroll       11.0
    mouseover     3.0
    blur          2.0
    Name: event, dtype: float64]
 ...
 [3 mousemove    8.0
    mouseover    4.0
    mousedown    1.0
    click        1.0
    Name: event, dtype: float64]
 [4 mousemove    8.0
    mouseover    2.0
    mousedown    1.0
    click        1.0
    Name: event, dtype: float64]
 [2 mousemove    4.0
    mouseover    2.0
    mousedown    1.0
    click        1.0
    Name: event, dtype: float64]]


In [27]:
inputs = keras.Input(shape=(784,))
inputs.shape

tab.transpose()
x_train = tab[1][0 : int(len(tab[1]/100*75))]
x_test = tab[1][int(len(tab[1]/100*75)) : len(tab[1])-1]
y_train = tab[0][0 : int(len(tab[0]/100*75))]
y_test = tab[0][int(len(tab[1]/100*75)) : len(tab[0])-1]

In [29]:
model = keras.Model()

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = model.fit(x_train, y_train, batch_size=64, epochs=2, validation_split=0.2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).